These data come from a survey conducted by INSEE, Budget des Familles, which are protected by the Statistical Secrecy Committee. To reproduce the analysis, one must request access to this data.

In [15]:
import pandas as pd

In [16]:
df_liste_menages = pd.read_csv('menagesEtudiant2.csv', index_col="IDENT_MEN")

In [17]:
df = pd.read_csv('../BDF/Csv/C05.csv', sep=";", index_col="IDENT_MEN")

In [18]:
df = df_liste_menages.merge(right=df, left_index=True, right_index=True)

In [19]:
column_averages = df.mean()

# Ensuite, vous pouvez créer une nouvelle ligne dans le DataFrame contenant les moyennes
average_row = pd.DataFrame(column_averages).T

# Maintenant, vous pouvez concaténer la ligne moyenne au DataFrame d'origine
df_with_average_row = pd.concat([df, average_row])

# Si vous voulez supprimer les lignes d'origine et ne garder que la ligne moyenne
df = df_with_average_row.iloc[-1:]

We need to remove the columns starting with 13 (taxes) and 14 (subsidies) which are not consumption. And also the columns total CTOT and pondmen (weight household) which are not useful.

In [20]:
columns_to_delete = df.columns[df.columns.str.startswith('C13') | df.columns.str.startswith('C14')]
df = df.drop(columns=columns_to_delete)
df = df.drop(columns=["CTOT", "pondmen"])

In [21]:
df_prop = df.divide(df.sum(axis=1), axis=0)


In [22]:
df_prop.rename(columns=lambda x: x.replace('C', ''), inplace=True)


We know have the proportion of each consumption area per household.

In [23]:
import eurostat

In [24]:
df_inflation_raw = eurostat.get_data_df(code="PRC_HICP_MANR", filter_pars={
                                    'geo': 'FR', 'startPeriod': "1997-01"})
df_inflation_raw = df_inflation_raw.rename(columns={'geo\TIME_PERIOD': 'geo'})

In [25]:
df_correspondances = pd.read_csv("../correspondances.csv", dtype="str", sep=";")

In [26]:
df_correspondances.set_index('colonnes de df_prop', inplace=True)

In [27]:
df_inflation = df_inflation_raw[df_inflation_raw['coicop'].str.startswith('CP')]

In [28]:
df_inflation['coicop'] = df_inflation['coicop'].str.replace('CP', '')


C:\Users\mdaversa\AppData\Local\Temp\ipykernel_11936\2914206753.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inflation['coicop'] = df_inflation['coicop'].str.replace('CP', '')


In [31]:
df_prop = df_prop.transpose()

In [29]:
df_prop.to_csv("budgetMoyenEtudiant.csv")